In [127]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [128]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [129]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [130]:
%%capture pwd
!pwd

In [ ]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

# Fig 1

## a

In [ ]:
df = pd.read_excel("raw/Figures_1_and_2.xlsx",sheet_name='£100 per week for 6 months').dropna()
df.columns=['wealth','current','government','dilnot']
df[['current','government','dilnot']]/=100.0

In [ ]:
f = "fig1a_100pw_6mo"
f1a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f1a).encode(
    x=alt.X(
        "wealth:Q",
        axis=alt.Axis(
            grid=False,
            titleAnchor="end",
            title="Wealth (£)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="right",
            titleX=400,
            titleY=-30,
            format="~s",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "current:Q",
        axis=alt.Axis(
            grid=True,
            title="Percentage of assets used to pay for care with costs of £100 per week for six months",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=20,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0%",
        ),
    ),
)
line2=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='government:Q')
line3=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='dilnot:Q')
label1=alt.Chart(pd.DataFrame([{'x':45000,'y':0.09}])).mark_text(text='Current system',align='left',color=colors['eco-mid-blue']).encode(x='x:Q',y='y:Q')
label2=alt.Chart(pd.DataFrame([{'x':330000,'y':0.03}])).mark_text(text='= Dilnot adjusted proposal',align='left',color=colors['eco-turquiose']).encode(x='x:Q',y='y:Q')
label3=alt.Chart(pd.DataFrame([{'x':110000,'y':0.03}])).mark_text(text='Government proposal',align='left',color=colors['eco-dot']).encode(x='x:Q',y='y:Q')
layer1 = (
    ((line1+line2+line3+label1+label2+label3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

## b

In [ ]:
df = pd.read_excel("raw/Figures_1_and_2.xlsx",sheet_name='£100 per week for 10 years').dropna()
df.columns=['wealth','current','government','dilnot']
df[['current','government','dilnot']]/=100.0

In [ ]:
f = "fig1b_100pw_10y"
f1b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f1b).encode(
    x=alt.X(
        "wealth:Q",
        axis=alt.Axis(
            grid=False,
            titleAnchor="end",
            title="Wealth (£)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="right",
            titleX=400,
            titleY=-30,
            format="~s",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "current:Q",
        axis=alt.Axis(
            grid=True,
            title="Percentage of assets used to pay for care with costs of £100 per week for ten years",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=20,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0%",
        ),
    ),
)
line2=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='government:Q')
line3=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='dilnot:Q')
label1=alt.Chart(pd.DataFrame([{'x':135000,'y':0.5}])).mark_text(text='Current system',align='left',color=colors['eco-mid-blue']).encode(x='x:Q',y='y:Q')
label2=alt.Chart(pd.DataFrame([{'x':420000,'y':0.3}])).mark_text(text='= Dilnot adjusted proposal',align='left',color=colors['eco-turquiose']).encode(x='x:Q',y='y:Q')
label3=alt.Chart(pd.DataFrame([{'x':200000,'y':0.3}])).mark_text(text='Government proposal',align='left',color=colors['eco-dot']).encode(x='x:Q',y='y:Q')
layer1 = (
    ((line1+line2+line3+label1+label2+label3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

# Fig 2

## a

In [ ]:
df = pd.read_excel("raw/Figures_1_and_2.xlsx",sheet_name='£500 per week for 6 months').dropna()
df.columns=['wealth','current','government','dilnot']
df[['current','government','dilnot']]/=100.0

In [ ]:
f = "fig2a_500pw_6mo"
f2a = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2a.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2a = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f2a).encode(
    x=alt.X(
        "wealth:Q",
        axis=alt.Axis(
            grid=False,
            titleAnchor="end",
            title="Wealth (£)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="right",
            titleX=400,
            titleY=-30,
            format="~s",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "current:Q",
        axis=alt.Axis(
            grid=True,
            title="Percentage of assets used to pay for care with costs of £500 per week for six months",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0%",
        ),
    ),
)
line2=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='government:Q')
line3=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='dilnot:Q')
label1=alt.Chart(pd.DataFrame([{'x':65000,'y':0.25}])).mark_text(text='Current system',align='left',color=colors['eco-mid-blue']).encode(x='x:Q',y='y:Q')
label2=alt.Chart(pd.DataFrame([{'x':420000,'y':0.08}])).mark_text(text='= Dilnot adjusted proposal',align='left',color=colors['eco-turquiose']).encode(x='x:Q',y='y:Q')
label3=alt.Chart(pd.DataFrame([{'x':200000,'y':0.08}])).mark_text(text='Government proposal',align='left',color=colors['eco-dot']).encode(x='x:Q',y='y:Q')
layer1 = (
    ((line1+line2+line3+label1+label2+label3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

## b

In [ ]:
df = pd.read_excel("raw/Figures_1_and_2.xlsx",sheet_name='£500 per week for 10 years').dropna()
df.columns=['wealth','current','government','dilnot']
df[['current','government','dilnot']]/=100.0

In [ ]:
f = "fig2b_500pw_10y"
f2b = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2b.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2b = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f2b).encode(
    x=alt.X(
        "wealth:Q",
        axis=alt.Axis(
            grid=False,
            titleAnchor="end",
            title="Wealth (£)",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="right",
            titleX=400,
            titleY=-30,
            format="~s",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "current:Q",
        axis=alt.Axis(
            grid=True,
            title="Percentage of assets used to pay for care with costs of £500 per week for ten years",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0%",
        ),
    ),
)
line2=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='government:Q')
line3=base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    )).encode(y='dilnot:Q')
label1=alt.Chart(pd.DataFrame([{'x':400000,'y':0.7}])).mark_text(text='Current system',align='left',color=colors['eco-mid-blue']).encode(x='x:Q',y='y:Q')
label2=alt.Chart(pd.DataFrame([{'x':340000,'y':0.30}])).mark_text(text='Dilnot adjusted proposal',align='left',color=colors['eco-turquiose']).encode(x='x:Q',y='y:Q')
label3=alt.Chart(pd.DataFrame([{'x':180000,'y':0.55}])).mark_text(text='Government proposal',align='left',color=colors['eco-dot']).encode(x='x:Q',y='y:Q')
layer1 = (
    ((line1+line2+line3+label1+label2+label3).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1